In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
%matplotlib inline

### Proprocess the diagnoses data

In: DIAGNOSES_ICD.csv from MIMIC3   

out: df with columns "SUBJECT_ID", "TIME","EVENTS"  

"TIME" is "ADMISSION_TIME" column joined from "ADMISSION.csv", with key "SUBJECT_ID" and "HADM_ID"

"EVENTS" is "ICD9_CODE" column

In [2]:
diag = pd.read_csv("./raw_mimic_data/DIAGNOSES_ICD.csv", dtype = str)

FileNotFoundError: File b'./raw_mimic_data/DIAGNOSES_ICD.csv' does not exist

In [52]:
diag.head(5)

ROW_ID SUBJECT_ID HADM_ID SEQ_NUM ICD9_CODE
0   1297        109  172335       1     40301
1   1298        109  172335       2       486
2   1299        109  172335       3     58281
3   1300        109  172335       4      5855
4   1301        109  172335       5      4254

In [53]:
print diag.shape[0]
print diag.ICD9_CODE.nunique()
print diag.SUBJECT_ID.nunique()

651047
6984
46520


In [54]:
#set wether to group icd 9 codes (first 3 string)
group_icd_codes = True

In [55]:
if group_icd_codes:
    diag["ICD9_CODE"] = diag["ICD9_CODE"].apply(lambda x: str(x)[:3])

In [56]:
#plenty of people diagnosed HF
print diag[diag["ICD9_CODE"] == "D428"].SUBJECT_ID.nunique()

0


In [57]:
#here I use first 3 digits to group diagonisis codes to avoid sparsity
diag_clean = diag.ix[:,["SUBJECT_ID", "ICD9_CODE"]]
diag_count = diag_clean.groupby("ICD9_CODE").count()

In [58]:
#124 codes have 5 instances or less. For now I will include them in the model but will check in the future
print diag_count.shape
print diag_count[diag_count.SUBJECT_ID < 5].shape

(943, 1)
(124, 1)


In [59]:
#now need to find a way to get the date and time 
#import boto
#join admission time as the time of the diagonsis
admission = pd.read_csv("./raw_mimic_data/ADMISSIONS.csv", dtype = str)
admission = admission.ix[:,["SUBJECT_ID", "HADM_ID", "ADMITTIME"]]

In [60]:
diag_w_time = diag.merge(admission, how = "inner", on=["SUBJECT_ID", "HADM_ID"])
diag_w_time.rename(columns = {"ADMITTIME": "TIME", "ICD9_CODE":"EVENTS"}, inplace = True )

In [61]:
diag_w_time.shape

(651047, 6)

In [62]:
diag_clean = diag_w_time.ix[:,["SUBJECT_ID", "EVENTS", "TIME"]]

In [63]:
#filter out all empty or null diag codes
diag_clean["EVENTS"] = diag_clean.EVENTS.str.strip()
diag_clean = diag_clean[~diag_clean.EVENTS.isnull()]
diag_clean = diag_clean[diag_clean.EVENTS != ""]
diag_clean.shape

(651047, 3)

In [64]:
#make sure no empty/null events
print diag_clean[diag_clean.EVENTS.apply(lambda x: len(x.strip())) == 0].shape[0]
print diag_clean[diag_clean.EVENTS.isnull()].shape[0]

0
0


In [65]:
#diagnosis data with time
diag_clean.head()

SUBJECT_ID EVENTS                 TIME
0        109    403  2141-09-18 10:32:00
1        109    486  2141-09-18 10:32:00
2        109    582  2141-09-18 10:32:00
3        109    585  2141-09-18 10:32:00
4        109    425  2141-09-18 10:32:00

### preprocess prescription data

In: PROCEDURES_ICD.csv from MIMIC3   

out: df with columns "SUBJECT_ID", "TIME","EVENTS"  

"TIME" is "STARTDATE" column

"EVENTS" is "DRUG" column as drug names

In [66]:
#now I will look at the prescription
pres = pd.read_csv("./raw_mimic_data/PRESCRIPTIONS.csv", dtype = str)

In [67]:
#again extremely skewed distribution, might need to group rare codes 
pres_clean = pres.ix[:, ["SUBJECT_ID", "STARTDATE", "DRUG"]]
drug_ct = pres_clean.groupby("DRUG").count()["SUBJECT_ID"]
drug_ct.describe()

count      4525.000000
mean        918.552486
std        6994.611320
min           1.000000
25%           1.000000
50%           4.000000
75%          50.000000
max      192993.000000
Name: SUBJECT_ID, dtype: float64

In [68]:
pres_clean.shape[0]

4156450

In [69]:
pres_clean.rename(columns = {"STARTDATE":"TIME", "DRUG":"EVENTS"},inplace = True)

In [70]:
#get rid of all null/empty observations
pres_clean["EVENTS"] = pres_clean.EVENTS.str.strip()
pres_clean["EVENTS"] = pres_clean.EVENTS.str.lower()

pres_clean = pres_clean[~pres_clean.EVENTS.isnull()]
pres_clean = pres_clean[pres_clean.EVENTS != ""]

#make sure no empty/null events
print pres_clean[pres_clean.EVENTS.apply(lambda x: len(x.strip())) == 0].shape[0]
print pres_clean[pres_clean.EVENTS.isnull()].shape[0]
print pres_clean.shape[0]

0
0
4156449


In [71]:
pres_clean.head()

SUBJECT_ID                 TIME          EVENTS
0          6  2175-06-11 00:00:00      tacrolimus
1          6  2175-06-11 00:00:00        warfarin
2          6  2175-06-11 00:00:00  heparin sodium
3          6  2175-06-11 00:00:00             d5w
4          6  2175-06-11 00:00:00      furosemide

### preprocess procedure data  

In: PROCEDURES_ICD.csv from MIMIC3   

out: df with columns "SUBJECT_ID", "TIME","EVENTS"  

"TIME" is "ADMISSION_TIME" column joined from "ADMISSION.csv", with key "SUBJECT_ID" and "HADM_ID"

"EVENTS" is "ICD9_CODE" column  

In [72]:
proc = pd.read_csv("./raw_mimic_data/PROCEDURES_ICD.csv", dtype = str)

In [73]:
proc_w_time = proc.merge(admission, how = "inner", on=["SUBJECT_ID", "HADM_ID"])
proc_w_time.rename(columns = {"ADMITTIME": "TIME", "ICD9_CODE":"EVENTS"}, inplace = True )

In [74]:
proc_w_time.shape[0]

240095

In [75]:
proc_clean = proc_w_time.ix[:,["SUBJECT_ID", "TIME","EVENTS"]]
proc_clean.head()

SUBJECT_ID                 TIME EVENTS
0      62641  2143-07-23 07:15:00   3404
1      62641  2143-07-23 07:15:00    326
2      62641  2143-07-23 07:15:00   3348
3       2592  2183-06-05 21:02:00   9671
4       2592  2183-06-05 21:02:00   3893

In [76]:
# no empty/null value in procesdure data set
print proc_clean[proc_clean.EVENTS.isnull()].shape[0]
print proc_clean[proc_clean.EVENTS.apply(lambda x: len(x.strip())) == 0].shape[0]

0
0


In [77]:
if group_icd_codes:
    proc_clean["EVENTS"] = proc_clean["EVENTS"].apply(lambda x: str(x)[:3])

In [78]:
proc_clean.TIME.min()

'2100-06-07 19:59:00'

### Preprocess Proceducre MV data
Another MIMIC procedure data. This data set contain detailed time for procedure

In: PROCEDURES_ICD.csv from MIMIC3   

out: df with columns "SUBJECT_ID", "TIME","EVENTS"  

"TIME" is "START_TIME" column 

"EVENTS" is "ITEM_description in "D_ITEMS.csv" joined by "ITEM_ID"

In [79]:
proc_events = pd.read_csv("./raw_mimic_data/PROCEDUREEVENTS_MV.csv", dtype = str)

In [80]:
proc_events.head()

ROW_ID SUBJECT_ID HADM_ID ICUSTAY_ID            STARTTIME  \
0    379      29070  115071     232563  2145-03-12 23:04:00   
1    380      29070  115071     232563  2145-03-12 23:04:00   
2    381      29070  115071     232563  2145-03-12 23:05:00   
3    382      29070  115071     232563  2145-03-12 23:36:00   
4    383      29070  115071     232563  2145-03-13 01:27:00   

               ENDTIME  ITEMID VALUE VALUEUOM  LOCATION      ...       \
0  2145-03-12 23:05:00  225401     1     None       NaN      ...        
1  2145-03-12 23:05:00  225454     1     None       NaN      ...        
2  2145-03-18 20:01:00  225792  8456     hour       NaN      ...        
3  2145-03-12 23:37:00  225402     1     None       NaN      ...        
4  2145-03-16 16:00:00  224560  5193      min  Right IJ      ...        

  ORDERCATEGORYNAME SECONDARYORDERCATEGORYNAME ORDERCATEGORYDESCRIPTION  \
0        Procedures                        NaN             Electrolytes   
1        Procedures                        NaN             Electrolytes   
2       Ventilation                        NaN                     Task   
3        Procedures                        NaN             Electrolytes   
4    Invasive Lines                        NaN                     Task   

  ISOPENBAG CONTINUEINNEXTDEPT CANCELREASON STATUSDESCRIPTION  \
0         0                  0            0   FinishedRunning   
1         0                  0            0   FinishedRunning   
2         1                  0            0   FinishedRunning   
3         0                  0            0   FinishedRunning   
4         1                  0            0   FinishedRunning   

  COMMENTS_EDITEDBY COMMENTS_CANCELEDBY COMMENTS_DATE  
0               NaN                 NaN           NaN  
1               NaN                 NaN           NaN  
2               NaN                 NaN           NaN  
3               NaN                 NaN           NaN  
4               NaN                 NaN           NaN  

[5 rows x 25 columns]

In [81]:
proc_events[proc_events.ORDERCATEGORYDESCRIPTION == "Electrolytes"].ORDERCATEGORYNAME.value_counts()

Procedures               66495
Imaging                  46310
Intubation/Extubation    12842
Significant Events        9913
Communication             8560
CRRT Filter Change         126
Name: ORDERCATEGORYNAME, dtype: int64

In [82]:
proc_events.ORDERCATEGORYDESCRIPTION.value_counts()

Electrolytes    144246
Task            113820
Name: ORDERCATEGORYDESCRIPTION, dtype: int64

In [83]:
proc_clean_MV = proc_events.ix[proc_events.ORDERCATEGORYDESCRIPTION == "Electrolytes", ["SUBJECT_ID", "STARTTIME", "ITEMID"]]
item_id = pd.read_csv("./raw_mimic_data/D_ITEMS.csv", dtype = str)
item_id = item_id.ix[:,["ITEMID", "LABEL"]]
proc_clean_MV = proc_clean_MV.merge(item_id, on = "ITEMID")

In [84]:
proc_clean_MV.rename(columns = {"STARTTIME" : "TIME","LABEL": "EVENTS"}, inplace = True)
proc_clean_MV = proc_clean_MV.drop(["ITEMID"], axis = 1)

In [85]:
print proc_clean_MV[proc_clean.EVENTS.isnull()].shape[0]
print proc_clean_MV[proc_clean.EVENTS.apply(lambda x: len(x.strip()) == 0)].shape[0]

proc_clean_MV["EVENTS"] = proc_clean_MV.EVENTS.str.strip()
proc_clean_MV["EVENTS"] = proc_clean_MV.EVENTS.str.lower()

/Users/yichenshen/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':
/Users/yichenshen/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  from ipykernel import kernelapp as app


0
0


In [86]:
proc_clean_MV.TIME.min()

'2100-07-03 02:52:00'

### Concatenate everything together 

In [87]:
use_mv = False

if use_mv:
    all_events_data = pd.concat([diag_clean,proc_clean_MV, pres_clean], axis = 0)
else:
    all_events_data = pd.concat([diag_clean,proc_clean, pres_clean], axis = 0)

In [88]:
all_events_data.shape[0]

5047591

In [89]:
from sklearn.utils import shuffle
#randomly shuffle the data so it breaks the sequence order with diag and procedure with same subject id and time
all_events_data = shuffle(all_events_data).reset_index(drop=True)

In [90]:
#turn medication, diag, procedure events into integer ids (by their alphabetical order)
all_events = all_events_data.EVENTS.unique()
all_events.sort()
index = np.arange(1,len(all_events)+1)
events_lookup = pd.DataFrame({"EVENTS":all_events, "EVE_INDEX":index})

In [91]:
if use_mv:
    events_lookup.to_csv("./cleaned_data/events_id_mv.csv")
else:
    events_lookup.to_csv("./cleaned_data/events_id.csv")

In [92]:
#merge all_events_data with event_index
all_events_data = all_events_data.merge(events_lookup, on = "EVENTS")

In [93]:
all_events_data = all_events_data.sort_values(by = ["SUBJECT_ID", "TIME"]).reset_index(drop=True)
all_events_data.head()

EVENTS SUBJECT_ID                 TIME  EVE_INDEX
0  neo*iv*ampicillin sodium         10  2103-06-28 00:00:00       3505
1           send 500mg vial         10  2103-06-28 00:00:00       4264
2                       966         10  2103-06-28 11:36:00       1041
3                       998         10  2103-06-28 11:36:00       1071
4                       V30         10  2103-06-28 11:36:00       1126

In [94]:
if use_mv:
    all_events_data.to_csv("./cleaned_data/all_events_data_mv.csv")
else:
    all_events_data.to_csv("./cleaned_data/all_events_data.csv")

### Explore rare words

In [95]:
eve_value_count = all_events_data.EVE_INDEX.value_counts()

In [96]:
eve_value_count[eve_value_count < 2].shape

(1055,)

In [49]:
eve_value_count.describe()

count      5478.000000
mean        921.429536
std        6448.069811
min           1.000000
25%           2.000000
50%          11.000000
75%         123.000000
max      193412.000000
Name: EVE_INDEX, dtype: float64